In [1]:
# Ref : https://huggingface.co/blog/chat-templates

In [4]:
import lightning.pytorch as pl
import torch
import torch.nn as nn
from datasets import load_dataset

In [5]:
ds = load_dataset(
    "skeskinen/TinyStories-Instruct-hf",
    cache_dir="/home/pranav-pc/projects/OpenTransformer/multiformer/data/downloads",
)

In [7]:
print(ds["train"][20]["text"])

Summary: Two dogs, Tom and Sam, fight over a bone and end up losing it when a man throws a stick at them. Sam takes the bone and runs away, leaving Tom sad and alone.
Features: Dialogue, BadEnding
Words: send, bone, fair
Story: 

Tom and Sam are dogs. They are friends. They like to play and run in the park. One day, they find a big bone. It smells good. They both want the bone.

"Let me have the bone!" Tom says. "I saw it first!"

"No, I want the bone!" Sam says. "I am bigger and stronger!"

They start to fight. They bark and bite. They do not share. They are not fair.

A man sees them. He is angry. He does not like noisy dogs. He picks up a stick. He throws the stick at them. He says, "Go away, you bad dogs!"

The stick hits Tom. It hurts. He drops the bone. Sam sees the bone. He grabs it. He runs away. He is happy.

Tom is sad. He is hurt. He has no bone. He has no friend. He wishes he was fair. He wishes he could send the bone to Sam and say sorry. But it is too late. Sam is gone. T

In [9]:
def convert_to_llama_chat_format(dataset):
    # Split the dataset into individual sections based on 'Features:'
    sections = dataset.split("Features:")
    conversation = ""

    for section in sections:
        if section.strip():
            # Extracting the relevant information from each section
            parts = section.split("\n")
            dialogue = parts[1].replace("Summary:", "").strip()
            story = "\n".join(parts[4:])

            # Constructing the LLAMA chat format
            conversation += f"<s>[INST] <<SYS>>\nSystem prompt: {dialogue}\n<</SYS>>\n"
            conversation += f"User prompt [/INST] {story}\nModel answer: {story}\n</s>\n"

    return conversation

In [10]:
print(convert_to_llama_chat_format(ds["train"][20]["text"]))

<s>[INST] <<SYS>>
System prompt: 
<</SYS>>
User prompt [/INST] 
Model answer: 
</s>
<s>[INST] <<SYS>>
System prompt: Words: send, bone, fair
<</SYS>>
User prompt [/INST] Tom and Sam are dogs. They are friends. They like to play and run in the park. One day, they find a big bone. It smells good. They both want the bone.

"Let me have the bone!" Tom says. "I saw it first!"

"No, I want the bone!" Sam says. "I am bigger and stronger!"

They start to fight. They bark and bite. They do not share. They are not fair.

A man sees them. He is angry. He does not like noisy dogs. He picks up a stick. He throws the stick at them. He says, "Go away, you bad dogs!"

The stick hits Tom. It hurts. He drops the bone. Sam sees the bone. He grabs it. He runs away. He is happy.

Tom is sad. He is hurt. He has no bone. He has no friend. He wishes he was fair. He wishes he could send the bone to Sam and say sorry. But it is too late. Sam is gone. Tom is alone. The end.
Model answer: Tom and Sam are dogs. Th

In [15]:
def convert_to_chat_format(text):
    text = "User:" + text
    text = text.replace("Story:", "Assistant:")
    return text


print(convert_to_chat_format(ds["train"][20]["text"]))

User:Summary: Two dogs, Tom and Sam, fight over a bone and end up losing it when a man throws a stick at them. Sam takes the bone and runs away, leaving Tom sad and alone.
Features: Dialogue, BadEnding
Words: send, bone, fair
Assistant: 

Tom and Sam are dogs. They are friends. They like to play and run in the park. One day, they find a big bone. It smells good. They both want the bone.

"Let me have the bone!" Tom says. "I saw it first!"

"No, I want the bone!" Sam says. "I am bigger and stronger!"

They start to fight. They bark and bite. They do not share. They are not fair.

A man sees them. He is angry. He does not like noisy dogs. He picks up a stick. He throws the stick at them. He says, "Go away, you bad dogs!"

The stick hits Tom. It hurts. He drops the bone. Sam sees the bone. He grabs it. He runs away. He is happy.

Tom is sad. He is hurt. He has no bone. He has no friend. He wishes he was fair. He wishes he could send the bone to Sam and say sorry. But it is too late. Sam i